# Import modules

In [1]:
import torch

from model import FeNeC

from configs.imagenet_subset_resnet import config
from utils.loader import FeNeCDataLoader

# Choose the device

In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

print(f"Using device: {device}")

Using device: mps


# Run the training and evalutating loop

For simplicity, you can pass extracted features from backbone with shapes as below to the model:
- X_train (num_train_samples, num_features)
- y_train (num_train_samples)
- X_test (num_test_samples, num_features)
- y_test (num_test_samples)

Here we are using our dataloader, but it requires the datasets to be stored in
the given data directory and the directory with dataset name (ex. "./data/tinyimagenet") Each task data need to be stored
in task_0.hdf5 files were 0 is number of task. In each of hdf5 files there are some keys specified. 
Look at utils/loader.py file for more information.

In [3]:
import torch
import numpy as np
from collections import defaultdict

num_tasks = 6
num_classes = 100
accuracies = []

data_loader = FeNeCDataLoader(
    num_tasks=num_tasks,
    dataset_name="FeCAM_seed_1993",
    load_covariances=True,
    load_prototypes=False,
    dataset_path="./data",
)
model = FeNeC(config, device=device)

# Matrix to hold per-task per-class accuracy
per_class_accuracy_matrix = np.zeros((num_tasks, num_classes))

for task_id in range(num_tasks):
    X_train, y_train, X_test, y_test, covariances, prototypes = data_loader.get_data(task_id)

    model.fit(X_train, y_train)
    predictions = model.predict(X_test)

    # Overall accuracy
    accuracy = (
        torch.sum((y_test.flatten().to(device) == predictions).int()) / X_test.shape[0]
    ).item()
    accuracies.append(accuracy)
    print(f"Task {task_id} - Accuracy: {accuracy}")

    # Per-class accuracy
    y_test_cpu = y_test.flatten().cpu()
    preds_cpu = predictions.cpu()

    for cls in range(num_classes):
        mask = y_test_cpu == cls
        if mask.sum().item() == 0:
            acc = np.nan  # No samples for this class
        else:
            correct = (preds_cpu[mask] == cls).sum().item()
            acc = correct / mask.sum().item()

        per_class_accuracy_matrix[task_id, cls] = acc

# Print average accuracy
print(f"\nAverage incremental accuracy: {sum(accuracies)/len(accuracies)}")

Warning!!! All values smaller than 0 were set to 0, bacuse tukey transformation can't handle negative values!


/Users/mrfishpl/Desktop/fecam-test/utils/metric_functions.py:89: UserWarning: The operator 'aten::linalg_svd' is not currently supported on the MPS backend and will fall back to run on the CPU. This may have performance implications. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/mps/MPSFallback.mm:14.)
  cov_inv = torch.linalg.pinv(covMatrices[i * f_num : (i + 1) * f_num, :]).to(


Task 0 - Accuracy: 0.873199999332428
Warning!!! All values smaller than 0 were set to 0, bacuse tukey transformation can't handle negative values!
Task 1 - Accuracy: 0.8353333473205566
Warning!!! All values smaller than 0 were set to 0, bacuse tukey transformation can't handle negative values!
Task 2 - Accuracy: 0.798285722732544
Warning!!! All values smaller than 0 were set to 0, bacuse tukey transformation can't handle negative values!
Task 3 - Accuracy: 0.7732499837875366
Warning!!! All values smaller than 0 were set to 0, bacuse tukey transformation can't handle negative values!
Task 4 - Accuracy: 0.7482222318649292
Warning!!! All values smaller than 0 were set to 0, bacuse tukey transformation can't handle negative values!
Task 5 - Accuracy: 0.725600004196167

Average incremental accuracy: 0.7923152148723602


In [4]:
per_class_accuracy_matrix.shape

(6, 100)

In [5]:
num_classes_task_0 = 50
num_classes_per_task = 10
bwt_list = [np.mean(per_class_accuracy_matrix[-1][:num_classes_task_0]) - np.mean(per_class_accuracy_matrix[0][:num_classes_task_0])]

for i in range(1, num_tasks-1):
    bwt_list.append(np.mean(per_class_accuracy_matrix[-1][i*num_classes_per_task + num_classes_task_0 - num_classes_per_task: i*num_classes_per_task + num_classes_task_0]) - np.mean(per_class_accuracy_matrix[i][i*num_classes_per_task + num_classes_task_0 - num_classes_per_task: i*num_classes_per_task + num_classes_task_0]))

bwt = np.mean(bwt_list) * 100
print(bwt)

-5.607999999999998


In [6]:
# data_from_kris = [[0.840],
# [0.8064,0.576],
# [0.793,0.536,0.601],
# [0.780,0.495,0.577,0.536],
# [0.7676,0.481,0.553,0.521,0.511],
# [0.7584,0.453,0.547,0.501,0.502,0.561]]

# bwt = 0
# for i in range(len(data_from_kris)-1):
#     bwt += data_from_kris[-1][i] - data_from_kris[i][-1]
# bwt /= 5

# print(bwt)

In [7]:

print(f"BWT ResNet CIFAR-100: {-6.051999999999998}")
print(f"BWT ResNet TinyImageNet: {-6.912033153448428}")
print(f"BWT ResNet Imagenet-Subset: {1}")
print(f"BWT ViT CIFAR: {-4.444444444444437}")
print(f"BWT ViT ImageNet-R: {6.7522401866910995}")

BWT ResNet CIFAR-100: -6.051999999999998
BWT ResNet TinyImageNet: -6.912033153448428
BWT ResNet Imagenet-Subset: 1
BWT ViT CIFAR: -4.444444444444437
BWT ViT ImageNet-R: 6.7522401866910995
